In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [2]:
# Step 1: Preprocess the data
# Load the dataset
data = pd.read_csv('data.csv')

In [3]:
# Parse dates and set it as the index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

In [4]:
# Normalize the price values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Price'].values.reshape(-1, 1))

In [13]:
# Split the dataset into training and testing sets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

In [11]:
# Generate input sequences and corresponding labels
def generate_sequences(data, sequence_length):
    X = []
    y = []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(X), np.array(y)

In [12]:
sequence_length = 100

In [14]:
X_train, y_train = generate_sequences(train_data, sequence_length)
X_test, y_test = generate_sequences(test_data, sequence_length)

In [19]:
# Step 2: Define the LSTM model
def create_model(optimizer='adam', loss='mean_squared_error', batch_size=16, units=128):
    model = Sequential()
    model.add(LSTM(units, input_shape=(sequence_length, 1)))
    model.add(Dense(1))
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    return model

In [20]:
# Step 3: Define the grid search parameters
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'loss': ['mean_squared_error', 'mse'],
    'batch_size': [16, 32, 64],
    'units': [64, 128, 256],
}

In [21]:
# Step 4: Create the grid search object
model = KerasClassifier(build_fn=create_model, verbose=0)
grid_search = GridSearchCV(model, param_grid, cv=5)

C:\Users\vikreth\AppData\Local\Temp/ipykernel_13788/3881638523.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


In [22]:
# Step 5: Fit the grid search object
grid_search.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# Step 6: Print the best parameters
print(grid_search.best_params_)